In [1]:
import trax
import numpy as np
from trax import layers as tl
from trax import fastmath


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
help(tl.tie_in)

Help on function tie_in in module trax.layers.research.efficient_attention:

tie_in(x, y)



In [103]:
help(fastmath.sort_key_val)

Help on function sort_key_val in module trax.fastmath.ops:

sort_key_val(*args, **kwargs)
    Sorts keys along dimension and applies same permutation to values.



In [83]:
a = np.random.normal(1,1,(2,2))
b = np.random.normal(4,1,(2,2))

In [100]:
a = np.array([5,6,7,3,4])
b = np.sort(a)
c = np.argsort(a)

In [89]:
b

array([3, 4, 5, 6, 7])

In [91]:
c

array([3, 4, 0, 1, 2])

In [134]:
t_n_hashes = 2
t_n_buckets = 4
t_n_seq = t_seqlen = 8
t_n_q = 3
n_v = 5

t_q = (np.array([(j % t_n_buckets) for j in range(t_n_seq)]) * np.ones((t_n_q, 1))).T
t_v = np.ones((t_n_seq, n_v))
t_buckets = np.array(
    [
        (j % t_n_buckets) + t_n_buckets * i
        for i in range(t_n_hashes)
        for j in range(t_n_seq)
    ]
)

In [138]:
def sort_buckets(buckets, q, v, n_buckets, n_hashes, seqlen, verbose=False):
    """ 
  Args:
    buckets: tensor of at least 2 dimension, 
    n_buckets: number of buckets in each hash table
    n_hashes: the number of hash tables    
    """
    if verbose:
        print("---sort_buckets--")
    ## Step 1
    ticker = np.arange(n_hashes*seqlen)
    if verbose:
        print("ticker", ticker.shape, ticker)
    ## Step 2
    buckets_and_t = seqlen * buckets + (ticker % seqlen)  # provided
    if verbose:
        print("buckets_and_t", buckets_and_t.shape, buckets_and_t)

    # Hash-based sort ("s" at the start of variable names means "sorted")
    # Step 3
    sbuckets_and_t, sticker = fastmath.sort_key_val(buckets_and_t, ticker, dimension=-1)
    if verbose:
        print("sbuckets_and_t", sbuckets_and_t.shape, sbuckets_and_t)
    if verbose:
        print("sticker", sticker.shape, sticker)
    # Step 4
    _, undo_sort = fastmath.sort_key_val(sticker, ticker, dimension=-1)
    if verbose:
        print("undo_sort", undo_sort.shape, undo_sort)

    # Step 5
    st = sticker % seqlen  # provided
    sq = np.take(q, st, axis = 0)
    sv = np.take(v, st, axis = 0)
    return sq, sv, sticker, undo_sort

In [139]:
t_sq, t_sv, t_sticker, t_undo_sort = sort_buckets(
    t_buckets, t_q, t_v, t_n_buckets, t_n_hashes, t_seqlen
)